In [7]:
import gurobipy as gp
from gurobipy import GRB
from math import floor
import pandas as pd

# Useful Variable Definitions

# Num 5 min interval in an hour
hrIntervals = 12 
# Num of decision variables in model
numDecisionVars = 7

# Create Parameters

# Default Paramters (Don't Change)

# Maximum discharge rate (MW)
maxDischargeRate = 2
# Maximum charge rate (MW)
maxChargeRate = -2
# Storage capacity of battery (MWh)
bStorage = 2
# Round Trip Efficiency of battery (unitless)
roundTripEfficiency = 0.81


# Input Paramters 

# Starting battery charge (MWh)
bStorage0 = 0

# Price predictions for energy ($/MWh)
df = pd.read_csv('prices.csv')
prices = df['Prices'].values.tolist()

try:

    # Create a new model
    m = gp.Model("batOptimser")

    # Create variables

    dispatchGen = {}
    dischargeBattery = {}
    chargeBattery = {}
    energyInStorage = {}
    dispatchCost = {}
    dispatchRevenue = {}
    dispatchProfit = {}
    
    for i, p in enumerate(prices):
        # Decision Variables
        dispatchGen[i] = m.addVar(lb=-GRB.INFINITY,name="dispatchGen")
        dischargeBattery[i] = m.addVar(lb=0,ub=maxDischargeRate,name="dischargeBattery")
        chargeBattery[i] = m.addVar(lb=maxChargeRate,ub=0,name="chargeBattery")
        energyInStorage[i] = m.addVar(lb=0,ub=bStorage,name="energyInStorage")
        
        dispatchCost[i] = m.addVar(lb=-GRB.INFINITY,name="dispatchCost")
        dispatchRevenue[i] = m.addVar(lb=-GRB.INFINITY,name="dispatchRevenue")
        dispatchProfit[i] = m.addVar(lb=-GRB.INFINITY,name="dispatchProfit",obj=-1) # Objective Function (negative to maximize)
        
        # Constraints
        if (i > 0):
            preserveEnergyInStorage = m.addConstr(energyInStorage[i] == (energyInStorage[i-1]) - (roundTripEfficiency*chargeBattery[i-1]/12) - (dischargeBattery[i-1]/12))
        
        batteryDispatchDefinition = m.addConstr(dispatchGen[i] == dischargeBattery[i] + chargeBattery[i])
        
        dispatchCostDefinition = m.addConstr(dispatchCost[i] == -1*chargeBattery[i]*p/12)
        dispatchRevenueDefinition = m.addConstr(dispatchRevenue[i] == dischargeBattery[i]*p/12)
        dispatchProfitDefinition = m.addConstr(dispatchProfit[i] == dispatchRevenue[i] - dispatchCost[i])
     
    # More Constraints
    EnergyInStorageCapacity0 = m.addConstr(energyInStorage[0] == bStorage0)
    
    # Preserve Energy In Storage Definition for final price (misses it in for loop)
    energyInStorage[len(prices)] = m.addVar(lb=0,ub=bStorage,name="energyInStorage")
    preserveEnergyInStorage = m.addConstr(energyInStorage[len(prices)] == (energyInStorage[len(prices)-1]) - (roundTripEfficiency*chargeBattery[len(prices)-1]/12) - (dischargeBattery[len(prices)-1]/12))

    # Optimize model
    m.setParam( 'OutputFlag', False ) # Suppresses Gurobi output
    m.optimize()

    
    # Stats + Debugging
    
    profit = 0
    cost = 0
    charges = 0
    chargeMW = 0
    discharges = 0
    dischargeMW = 0
    waiting = 0
    interval = -1
    
    for i,v in enumerate(m.getVars()):
        if (i % numDecisionVars == 0):
                # Counts current time interval
                interval += 1

        if (v.varName == "energyInStorage" and interval == 1):
            # Gets next battery charge based on current optimal action 
            nxtBatCharge = v.x
    
        if (v.varName == "dispatchGen"):
            # Model data collection
            if (v.x > 0):
                if (interval == 0):
                    nxtAction = v.x
                currAction = "DISCHARGE"
                profit += v.x*prices[floor(i/numDecisionVars)]/hrIntervals
                discharges += 1
                dischargeMW += v.x
            elif (v.x < 0):
                if (interval == 0):
                    nxtAction = v.x
                currAction = "CHARGE"
                profit += v.x*prices[floor(i/numDecisionVars)]/hrIntervals
                cost += -v.x*prices[floor(i/numDecisionVars)]/hrIntervals
                charges += 1
                chargeMW += -v.x 
            else:
                if (interval == 0):
                    nxtAction = v.x
                currAction = "DO NOTHING"
                waiting += 1
                
            # DEBUGGING FOR DECISION VARIABLES -> Iteration number, price and action assigned
            #print("\nPrice Interval %g, Price: %.4g\nAction = %s" % (interval+1, prices[interval], currAction))

        # DEBUGGING FOR DECISIONS VARIABLES -> Variable names and values assigned
        #print('%s %g' % (v.varName, v.x))

    # OPTIMAL SOLUTION STATS    
    
    print('Model objective value: %.4g' % m.objVal)
    print("Actual Profit = $%.4g -> %.2g%% Profit" % (profit, 100*profit/cost))
    print("Charged %.4g MW over %d charges (Lost %.4g MW due to battery inefficiency)" % (chargeMW, charges, chargeMW-(roundTripEfficiency*chargeMW)))
    print("Discharges %.4g MW over %s discharges" % (dischargeMW, discharges))
    print("Did nothing during %d time intervals" % (waiting))
    print("Min price: $%.4g, Max price: $%.4g" % (min(prices), max(prices)))
    
    if (nxtAction < 0):
        print("\nBattery action this time interval: CHARGE %g MW" % (-nxtAction/12))
    elif (nxtAction > 0):
        print("\nBattery action this time interval: DISCHARGE %g MW" % (nxtAction/12))
    else:
        print("\nBattery action this time interval: DO NOTHING")
              
    print("Battery charge next time interval: %.5g MWh" % (nxtBatCharge))
    
    # nxtAction = Optimal solution for next battery action (MWh)
    # CHARGE < 0, DISCHARGE > 0, DO NOTHING = 0
    # nxtBatCharge = Bat charge next time interval if nxtAction is used and battery efficiency accounted for


except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ': ' + str(e))

except AttributeError:
    print('Encountered an attribute error')

Model objective value: -18.75
Actual Profit = $18.75 -> 27% Profit
Charged 49.88 MW over 26 charges (Lost 9.477 MW due to battery inefficiency)
Discharges 40.4 MW over 23 discharges
Did nothing during 239 time intervals
Min price: $0, Max price: $29.77

Battery action this time interval: CHARGE 0.166667 MW
Battery charge next time interval: 0.135 MWh


In [8]:
import optimizer_module as om
import pandas as pd

In [9]:
df = pd.read_csv('prices.csv')
prices = df['Prices'].values.tolist()

nxtAction, nxtBatCharge = om.optimize(prices, 0)
print(nxtAction, nxtBatCharge)

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (win64)
Optimize a model with 1441 rows, 2017 columns and 4031 nonzeros
Model fingerprint: 0x626fdf74
Coefficient statistics:
  Matrix range     [7e-02, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [2e+00, 2e+00]
  RHS range        [0e+00, 0e+00]
Presolve removed 1154 rows and 1157 columns
Presolve time: 0.01s
Presolved: 287 rows, 860 columns, 1146 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -9.4100243e+02   6.246121e+03   0.000000e+00      0s
      26   -1.8749766e+01   0.000000e+00   0.000000e+00      0s

Solved in 26 iterations and 0.01 seconds
Optimal objective -1.874976574e+01
-2.0 0.135
